In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/concepts.csv
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/__init__.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/mcts_inference_server.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/templates.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/base_gateway.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/relay.py
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evaluation/core/kaggle_evaluation.proto
/kaggle/input/um-game-playing-strength-of-mcts-variants/kaggle_evalua

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import KNNImputer
from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor, LocalOutlierFactor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from catboost import CatBoostRegressor
import os
import pyarrow as pa
import pyarrow.parquet as pq
import polars as pl
import kaggle_evaluation.mcts_inference_server

In [15]:
seed=42
np.random.seed(seed)


In [3]:
# Paths
train_path = "/kaggle/input/um-game-playing-strength-of-mcts-variants/train.csv"



In [4]:
# Load data
train_data = pd.read_csv(train_path)

In [5]:
train_data

,Id,GameRulesetName,agent1,agent2,Properties,Format,Time,Discrete,Realtime,Turns,...,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,num_wins_agent1,num_draws_agent1,num_losses_agent1,utility_agent1
0,0,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-ProgressiveHistory-0.6-Random200-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",4,0,11,-0.466667
1,1,00Y,MCTS-ProgressiveHistory-0.1-MAST-false,MCTS-UCB1GRAVE-0.6-NST-true,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
2,2,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.1-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",7,0,8,-0.066667
3,3,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1-0.6-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
4,4,00Y,MCTS-ProgressiveHistory-0.1-MAST-true,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,1,298.07,18877.17,Goal: Connect all three edge colors with a sin...,"(game ""00'Y'"" (players 2) (equipment { (board ...",5,0,10,-0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233229,233229,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-NST-false,MCTS-ProgressiveHistory-1.41421356237-Random20...,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",2,0,13,-0.733333
233230,233230,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1-0.6-MAST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",9,1,5,0.266667
233231,233231,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-false,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",11,3,1,0.666667
233232,233232,Zuz_Mel_7x7,MCTS-UCB1Tuned-1.41421356237-Random200-false,MCTS-UCB1GRAVE-1.41421356237-NST-true,1,1,1,1,0,1,...,0,0,157.52,157174.58,7x7 board. 24 pieces per player. Pieces begin ...,"(game ""Zuz Mel (7x7)"" (players 2) (equipment {...",24,2,4,0.666667


In [6]:

# Drop columns with 95% or more identical values and columns with all null values
def drop_column_name(df):
    drop_columns = []
    for column in df.columns:
        if df[column].value_counts().max() / len(df) >= 0.95 or df[column].isnull().all():
            drop_columns.append(column)
    return drop_columns

drop_column = drop_column_name(train_data)
drop_column.extend(["Id", "EnglishRules", "LudRules", "num_wins_agent1", "num_draws_agent1", "num_losses_agent1"])

drop_column.extend(['GameRulesetName', 'agent1', 'agent2'])

In [16]:
# Prepare train dataset
train = train_data.drop(columns=drop_column, errors='ignore')
# le1 = LabelEncoder()
# le2 = LabelEncoder()
# le3 = LabelEncoder()
# train["GameRulesetName"] = le1.fit_transform(train["GameRulesetName"])
# train["agent1"] = le2.fit_transform(train["agent1"])
# train["agent2"] = le3.fit_transform(train["agent2"])

# Split data
X = train.drop('utility_agent1', axis=1)  # Features
y = train['utility_agent1']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=seed)





In [17]:
# parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
#               'objective':['reg:linear'],
#               'learning_rate': [.01, 0.05, .07], #so called `eta` value
#               'max_depth': [5, 6, 7],
#               'min_child_weight': [4],
#               'silent': [1],
#               'subsample': [0.7],
#               'colsample_bytree': [0.7],
#               'n_estimators': [500]}

In [23]:
# model = XGBRegressor(n_estimators=1000, learning_rate=0.05,seed=seed,objective='reg:squarederror')

In [24]:
# xgb_grid = GridSearchCV(model,
#                         parameters,
#                         cv = 5,
#                         n_jobs = 5,
#                         verbose=True)

In [25]:
# xgb_grid.fit(X_train,
#          y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:37:31] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:37:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:37:31] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:37:31] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [23:37:31] WARNING: /workspace/src/objective/regress

GridSearchCV(cv=5,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=0.05, m...
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=1000,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=5,
             param_grid={'colsample_bytree': [0.7],
                         'learning_rate': [0.01, 0.05, 0.07],
                         'max_depth': [5, 6, 7], 'min_child_weight': [4],
                         'n_estimators': [500], 'nthread': [4],
                         'objective': ['reg:linear'], 'silent': [1],
                         'subsample': [0.7]},
             verbose=True)

In [26]:
# print(xgb_grid.best_score_)
# print(xgb_grid.best_params_)

0.44323698002363815
{'colsample_bytree': 0.7, 'learning_rate': 0.05, 'max_depth': 6, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:linear', 'silent': 1, 'subsample': 0.7}


In [ ]:
# model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6,colsample_bytree=0.7)

In [9]:
# Define model training function
def train_model(X_train, y_train):
    model = XGBRegressor(n_estimators=1000, learning_rate=0.05, max_depth=6,colsample_bytree=0.7)
    model.fit(X_train, y_train)
    print(222)
    return model


def clean_data(test_df):
    """
    Cleans the test dataframe by converting all applicable columns to numeric and handling errors.
    """
    # Drop columns that are not required
    test_df = test_df.drop(columns=drop_column, errors='ignore')
    
    # Ensure the columns are present
#     required_columns = ['GameRulesetName', 'agent1', 'agent2']
#     for col in required_columns:
#         if col not in test_df.columns:
#             test_df[col] = -1  # Assign a default value or handle missing columns appropriately

    # Transform categorical data using pre-fitted LabelEncoders
#     try:
#         test_df["GameRulesetName"] = le1.transform(test_df["GameRulesetName"])
#     except KeyError:
#         test_df["GameRulesetName"] = -1  # Assign a default value for unseen categories
    
#     try:
#         test_df["agent1"] = le2.transform(test_df["agent1"])
#     except KeyError:
#         test_df["agent1"] = -1
    
#     try:
#         test_df["agent2"] = le3.transform(test_df["agent2"])
#     except KeyError:
#         test_df["agent2"] = -1

    # Fill missing values
    test_df = test_df.fillna(0)
    print(333)
    return test_df

In [10]:
counter = 0
model = None  # Initialize model as None globally

def predict(test: pl.DataFrame, sample_sub: pl.DataFrame):
    global counter, model, X, y  # Declare all global variables at the start
    print(1)
    test_df = test.to_pandas()

    # Clean the data
    test_df = clean_data(test_df)
    print(2)

    # Train the model only once
    if counter == 0:
        model = train_model(X, y)
        counter += 1

    print(3)

    # Return NaN values on failure
    try:
        predictions = model.predict(test_df)
    except Exception as e:
        print(f"Error during prediction: {e}")
        return None

    # Summation step: add a constant value to the predictions
    
    print(predictions)
    print(type(predictions))

    # Prepare the final submission format
    submission = sample_sub.with_columns(pl.Series("utility_agent1", predictions))
    print(submission)
    print(type(submission))

    return submission


In [11]:
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )    
    )

1
333
2
222
3
[-0.05277699 -0.05277699 -0.05277699]
<class 'numpy.ndarray'>
shape: (3, 2)
┌────────┬────────────────┐
│ Id     ┆ utility_agent1 │
│ ---    ┆ ---            │
│ i64    ┆ f32            │
╞════════╪════════════════╡
│ 233235 ┆ -0.052777      │
│ 233236 ┆ -0.052777      │
│ 233234 ┆ -0.052777      │
└────────┴────────────────┘
<class 'polars.dataframe.frame.DataFrame'>
